In [ ]:
import cv2
import numpy as np

seed_points = []
select_done = False

def resize_image(image, width=640, height=480):
    return cv2.resize(image, (width, height), interpolation=cv2.INTER_AREA)

def mouse_callback(event, x, y, flags, param):
    global seed_points, select_done
    
    if event == cv2.EVENT_LBUTTONDOWN:
        seed_points.append((y, x))
        print(f"Seed point selected: ({x}, {y}) -> Image coordinates: ({y}, {x})")
        
        cv2.circle(param, (x, y), 3, (0, 0, 255), -1)
        cv2.imshow("Click left mouse button to select seed points (Press ESC to confirm)", param)

def region_growing(image, seed_points, threshold):
    if len(image.shape) != 2:
        raise ValueError("Region growing algorithm only supports single-channel grayscale images!")
    
    if not seed_points:
        raise ValueError("No seed points selected!")
    
    height, width = image.shape
    mask = np.zeros_like(image, dtype=np.uint8)  
    visited = np.zeros((height, width), dtype=bool)  
    directions = [
        (-1, 0),   
        (1, 0),    
        (0, -1),  
        (0, 1),  
        (1, 1),
        (-1, 1),
        (1, -1),
        (-1, -1)
    ]
    
    for (x, y) in seed_points:
        if x < 0 or x >= height or y < 0 or y >= width:
            print(f"Warning: Seed point ({x}, {y}) is out of image range, skipped")
            continue
        if visited[x, y]:
            continue  
        queue = [(x, y)]
        visited[x, y] = True
        mask[x, y] = 255  
        
        while queue:
            cx, cy = queue.pop(0)  
            for dx, dy in directions:
                nx, ny = cx + dx, cy + dy
                if 0 <= nx < height and 0 <= ny < width and not visited[nx, ny]:
                    if abs(int(image[nx, ny]) - int(image[cx, cy])) <= threshold:
                        visited[nx, ny] = True
                        mask[nx, ny] = 255 
                        queue.append((nx, ny)) 
    return mask

if __name__ == "__main__":
    image_path = 
    bgr_img = cv2.imread(image_path)
    if bgr_img is None:
        raise ValueError(f"Failed to read image, please check if the path is correct: {image_path}")

    bgr_img_resized = resize_image(bgr_img)
    
    print("===== Seed Point Selection Phase =====")
    print("Operation Instructions:")
    print("1. Click the left mouse button in the pop-up image window to select seed points (multiple selection allowed)")
    print("2. Press the ESC key to confirm and continue the program after selection")
    
    select_window_name = "Click left mouse button to select seed points (Press ESC to confirm)"
    cv2.namedWindow(select_window_name, cv2.WINDOW_NORMAL)
    cv2.resizeWindow(select_window_name, 640, 480)
    cv2.setMouseCallback(select_window_name, mouse_callback, bgr_img_resized.copy())
    
    cv2.imshow(select_window_name, bgr_img_resized)
    
    while True:
        key = cv2.waitKey(1) & 0xFF
        if key == 27:
            break
    
    cv2.destroyWindow(select_window_name)
    
    if not seed_points:
        print("Error: No seed points selected, program exits!")
        cv2.destroyAllWindows()
        exit(1)
    
    print(f"\n===== List of selected seed points: {seed_points} =====")
    
    bgr_img = bgr_img_resized
    grayimg = cv2.cvtColor(bgr_img, cv2.COLOR_BGR2GRAY)
    
    median_kernel_size = 3
    m = 1326.395647    
    k = 0.00128633  
    c = -1368.862781      
    
    img = cv2.medianBlur(grayimg, median_kernel_size)
    
    g = m * np.exp(k * img.astype(np.float32)) + c
    g = np.maximum(g, 0)
    g = np.minimum(g, 255)
    processed_img = np.clip(g, 0, 255).astype(np.uint8)
    
    growth_threshold = 1  
    print(f"\n===== Start executing region growing algorithm (Threshold: {growth_threshold}) =====")
    
    mask_original = region_growing(grayimg, seed_points, growth_threshold)
    mask_median = region_growing(img, seed_points, growth_threshold)
    mask_processed = region_growing(processed_img, seed_points, growth_threshold)
    
    cp = cv2.Canny(processed_img, 70, 190)
    ycp = cv2.Canny(img, 70, 190)
    bgr_gray = cv2.cvtColor(bgr_img, cv2.COLOR_BGR2GRAY)
    bgr_cp = cv2.Canny(bgr_gray,70, 190)
    
    img_resized = resize_image(img)
    processed_img_resized = resize_image(processed_img)
    cp_resized = resize_image(cp)
    ycp_resized = resize_image(ycp)
    bgr_cp_resized = resize_image(bgr_cp)
    
    mask_original_resized = resize_image(mask_original)
    mask_median_resized = resize_image(mask_median)
    mask_processed_resized = resize_image(mask_processed)
    
    cv2.imshow("origin p (640x480)", bgr_img)
    cv2.imshow("ming (640x480)", img_resized)
    cv2.imshow("pre p (640x480)", processed_img_resized)
    cv2.imshow("cp (640x480)", cp_resized)
    cv2.imshow("ycp (640x480)", ycp_resized)
    cv2.imshow("bgr_cp (640x480)", bgr_cp_resized)
    
    cv2.imshow("region_original (640x480)", mask_original_resized)
    cv2.imshow("region_median (640x480)", mask_median_resized)
    cv2.imshow("region_processed (640x480)", mask_processed_resized)

    print("\n===== All windows are displayed =====")
    print("Press ESC key to close all windows and exit the program")
    while True:
        key = cv2.waitKey(1) & 0xFF
        if key == 27:
            break
    
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    cv2.destroyAllWindows()
    cv2.waitKey(1)